从两个渠道下载上市公司年报

# 深交所和上交所股票代码的区别
>股票代码用数字表示股票的不同含义。股票代码除了区分各种股票，也有其潜在的意义。沪市股票买卖的代码是以600或601打头，如：运盛实业：股票代码是600股票代码767。中国国航：股票代码是601111。B股买卖的代码是以900打头，如：上电B股：代码是900901。深市A股票买卖的代码是以000打头，如：顺鑫农业：股票代码是000860。B股买卖的代码是以200打头，如：深中冠B股，代码是200018。

>沪市新股申购的代码是以730打头。如：中信证券：申购的代码是730030。深市新股申购的代码与深市股票买卖代码一样，如：中信证券在深市市值配售代码是003030。配股代码，沪市以700打头，深市以080打头。如：运盛实业配股代码是700767。深市草原兴发配股代码是080780。中小板股票代码以002打头，如：东华合创股票代码是002065。创业板股票代码以300打头，如：探路者股票代码是：300005


# 深圳证券交易所：
http://www.szse.cn/disclosure/listed/notice/index.html


# 上海深圳交易所
http://www.sse.com.cn/disclosure/listedinfo/announcement/





In [100]:
import requests
import re
def get_url_by_code_szse(code):
    download_url_prefix = 'http://disc.static.szse.cn/download'
    
    url = 'http://www.szse.cn/api/disc/announcement/annList'
    payload = {"seDate":["",""],"stock":["000001"],"channelCode":["listedNotice_disc"],"bigCategoryId":["010301"],"pageSize":30,"pageNum":1}
    headers = {'content-type': 'application/json',
              'Host': 'www.szse.cn',
    'Origin': 'http://www.szse.cn',
    'Referer': 'http://www.szse.cn/disclosure/listed/notice/index.html',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'      
              }
    payload['stock']=[code]
    payload['pageNum']=1
    r = requests.post(url, json=payload)
    ret = r.json()
    count = ret['announceCount']
    data_list = ret['data']
    # 由于限制是30每页，总数超过30，要第二次请求
    if(count>30):
        payload['pageNum']=2
        r = requests.post(url, json=payload)
        ret = r.json()
        data_list.extend(ret['data'])
        
    year_report_map={}
    for d in data_list:
        fileName = d['title']
        if(fileName.endswith('摘要')):
            continue
        if(not(fileName.endswith('年报') or fileName.endswith('年度报告'))):
            continue
        year = re.findall('\d{4}',fileName)[0]
        year_report_map[year]=download_url_prefix+d['attachPath']
        print(d['secName'][0],d['title'],d['attachPath'])
    return year_report_map


get_url_by_code_szse('000002')
        

In [106]:
from datetime import datetime,timedelta
# 上海证券交易所
def get_url_by_code_sse(code):
    url = 'http://query.sse.com.cn/security/stock/queryCompanyStatementNew.do'
    params={
        'jsonCallBack': 'jsonpCallback59529',
        'isPagination': 'true',
        'productId': '600795',
        'keyWord': '',
        'isNew': '1',
        'reportType2': 'DQBG',
        'reportType': 'YEARLY',
        'beginDate': '2002-04-13',
        'endDate': '2019-04-13',
        'pageHelp.pageSize': 25,
        'pageHelp.pageCount': 50,
        'pageHelp.pageNo': 1,
        'pageHelp.beginPage': 1,
        'pageHelp.cacheSize': 1,
        'pageHelp.endPage': 5,
        '_': 1555167217524
    }
    headers = {'Referer': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    data_list = []
    params['productId']=code
    beginDate=datetime.strptime('2003-01-01','%Y-%m-%d')
    _endDate= datetime.now()
    _3years = timedelta(days=365*3)
    while beginDate< _endDate:
        params['beginDate']=beginDate.strftime('%Y-%m-%d')
        beginDate += _3years
        params['endDate']=beginDate.strftime('%Y-%m-%d')
        r = requests.get(url,headers=headers,params=params)
        json_str = r.text[r.text.index('(')+1:-1]
        json_obj = json.loads(json_str)
        data_list.extend(json_obj['result'])


    year_report_map={}
    for d in data_list:
        fileName = d['title']
        if(fileName.endswith('摘要')):
            continue
        if(not(fileName.endswith('年报') or fileName.endswith('年度报告'))):
            continue
        print(d['bulletin_Year'],d['title'],d['URL'])
        year_report_map[d['bulletin_Year']]='http://www.sse.com.cn'+d['URL']
        
    return year_report_map


get_url_by_code_sse('600795')

2004 国电电力年报 /disclosure/listedinfo/announcement/c/2005-03-03/600795_2004_n.pdf
2003 国电电力年报 /disclosure/listedinfo/announcement/c/2004-02-24/600795_2003_n.pdf
2007 国电电力年报 /disclosure/listedinfo/announcement/c/2008-02-05/600795_2007_n.pdf
2006 国电电力年报 /disclosure/listedinfo/announcement/c/2007-03-08/600795_2006_n.pdf
2005 国电电力年报 /disclosure/listedinfo/announcement/c/2006-04-08/600795_2005_n.pdf
2010 国电电力年报 /disclosure/listedinfo/announcement/c/2011-03-31/600795_2010_n.pdf
2009 国电电力年报 /disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf
2008 国电电力年报 /disclosure/listedinfo/announcement/c/2009-04-18/600795_2008_n.pdf
2013 国电电力年报 /disclosure/listedinfo/announcement/c/2014-03-24/600795_2013_n.pdf
2012 国电电力年报 /disclosure/listedinfo/announcement/c/2013-03-29/600795_2012_n.pdf
2011 国电电力年报 /disclosure/listedinfo/announcement/c/2012-03-30/600795_2011_n.pdf
2016 国电电力2016年年度报告 /disclosure/listedinfo/announcement/c/2017-04-11/600795_2016_n.pdf
2015 国电电力年报 /disclosure/listedinfo/announceme

{'2003': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2004-02-24/600795_2003_n.pdf',
 '2004': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2005-03-03/600795_2004_n.pdf',
 '2005': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2006-04-08/600795_2005_n.pdf',
 '2006': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2007-03-08/600795_2006_n.pdf',
 '2007': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2008-02-05/600795_2007_n.pdf',
 '2008': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2009-04-18/600795_2008_n.pdf',
 '2009': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf',
 '2010': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2011-03-31/600795_2010_n.pdf',
 '2011': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2012-03-30/600795_2011_n.pdf',
 '2012': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2013-03-29/600795_2012_n.pdf',


In [109]:
url = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf'
r = requests.get(url, stream=True)

filename = './600795_2009.pdf';
with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=102400):
        fd.write(chunk)
        
import subprocess
try:
    out_bytes = subprocess.check_output(['which','pdf2txt.py'])
    print(out_bytes)
    pdf2txt=out_bytes.decode("utf-8").replace('\n','')
    subprocess.run(["python", pdf2txt, "-o",filename.replace('pdf','txt') ,filename])
except subprocess.CalledProcessError as e:
    out_bytes = e.output       # Output generated before error
    code      = e.returncode   # Return code
    



b'/Users/lcz/anaconda3/bin/pdf2txt.py\n'
